In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#Initial load 
littlerock_table_id = "opendatadbt.311.littlerock311"

littlerock311 = requests.get("https://data.littlerock.gov/resource/2x6n-j9fb.json?$limit=10000&$where=ticket_created_date_time>='2021-03-22T00:00:00.000'") 
littlerockjson = littlerock311.json()
littlerock_df_parsed = littlerock_df['geocoded_column'].apply(pd.Series)
littlerock_df_parsed = littlerock_df_parsed.drop(littlerock_df_parsed.columns[[0,1]], axis=1)
littlerock_df_cleaned = pd.concat([littlerock_df.drop(['geocoded_column'], axis=1), littlerock_df_parsed], axis=1)
littlerock_df_cleaned
cols = ['ticket_id','ticket_created_date_time','ticket_closed_date_time','issue_type','issue_sub_category','human_address','ticket_status','latitude','longitude']
littlerock_df_cleaned = pd.DataFrame(littlerock_df_cleaned, columns = cols)

In [3]:
#Incremental load
littlerock_table_id = "opendatadbt.311.littlerock311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(ticket_created_date_time), max(ticket_closed_date_time)
FROM `opendatadbt.311.littlerock311`
limit 10
"""

littlerockmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
littlerockmaxopendatedate = str(littlerockmaxopendatedataframe['f0_'][0])

littlerockmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
littlerockmaxclosedate = str(littlerockmaxclosedatedataframe['f1_'][0])

In [4]:
littlerockmaxclosedate

'2021-03-23T02:43:00.000'

In [6]:

littlerock_table_id = "opendatadbt.311.littlerock311"

littlerock311 = requests.get("https://data.littlerock.gov/resource/2x6n-j9fb.json?$limit=10000&$where=ticket_created_date_time>'"+littlerockmaxopendatedate+"' OR ticket_closed_date_time between '" +littlerockmaxclosedate+ "' AND '" + current_datetime + "'") 
littlerockjson = littlerock311.json()
littlerock_df = pd.DataFrame(littlerockjson)
littlerock_df_parsed = littlerock_df['geocoded_column'].apply(pd.Series)
littlerock_df_parsed = littlerock_df_parsed.drop(littlerock_df_parsed.columns[[0,1]], axis=1)
littlerock_df_cleaned = pd.concat([littlerock_df.drop(['geocoded_column'], axis=1), littlerock_df_parsed], axis=1)
littlerock_df_cleaned
cols = ['ticket_id','ticket_created_date_time','ticket_closed_date_time','issue_type','issue_sub_category','human_address','ticket_status','latitude','longitude']
littlerock_df_cleaned = pd.DataFrame(littlerock_df_cleaned, columns = cols)

In [7]:
littlerock_df_cleaned

,ticket_id,ticket_created_date_time,ticket_closed_date_time,issue_type,issue_sub_category,human_address,ticket_status,latitude,longitude
0,21-00000894,2021-01-05T10:18:00.000,2021-03-23T13:21:00.000,Code Violations,High Grass & Weeds,"{""address"": ""9001 N RODNEY PARHAM RD"", ""city"":...",Closed,34.7570213,-92.37490948
1,21-00000899,2021-01-05T10:21:00.000,2021-03-23T13:23:00.000,Code Violations,Trash / Rubbish / Debris on Premise,"{""address"": ""9001 N RODNEY PARHAM RD"", ""city"":...",Closed,34.7570213,-92.37490948
2,21-00004313,2021-01-14T16:39:00.000,2021-03-23T08:36:00.000,Public Works,Street Name Signs - Repair and Replace,"{""address"": ""PINE VALLEY RD & PINE VALLEY RD"",...",Closed,34.78207004,-92.34902886
3,21-00004351,2021-01-15T08:20:00.000,2021-03-24T14:20:00.000,Code Violations,Trash / Rubbish / Debris on Premise,"{""address"": ""7701 IMPALA DR"", ""city"": ""LITTLE ...",Closed,34.65638419,-92.36413768
4,21-00007744,2021-01-26T16:17:00.000,2021-03-23T15:49:00.000,Code Violations,Housing / Rental Code Violations (Mobile Homes),"{""address"": ""9500 S HEIGHTS RD"", ""city"": ""LITT...",Closed,34.66242176,-92.3324986
...,...,...,...,...,...,...,...,...,...
1368,21-00026870,2021-03-24T16:15:00.000,2021-03-24T16:15:00.000,Trash Complaints,On-Call / Bulky Items / Special Pick-Up Request,"{""address"": ""3801 W CHARLES BUSSEY AVE"", ""city...",Open,34.73297089,-92.31635821
1369,21-00026237,2021-03-23T15:32:00.000,2021-03-24T09:51:00.000,Trash Complaints,On-Call / Bulky Items / Special Pick-Up Request,"{""address"": ""9618 ITHICA DR"", ""city"": ""LITTLE ...",Open,34.66241617,-92.32353538
1370,21-00025832,2021-03-23T09:10:00.000,2021-03-23T10:47:00.000,Trash Complaints,Missed Yard Waste Pick-Up,"{""address"": ""100 S PLAZA DR"", ""city"": ""LITTLE ...",Open,34.752237,-92.34762249
1371,21-00026652,2021-03-24T12:30:00.000,2021-03-24T12:30:00.000,Animal Services,Stray Animal - Running at Large,"{""address"": ""2705 JEFFERSON ST"", ""city"": ""LITT...",Open,34.72688332,-92.32604607


In [8]:
littlerock_table_id = "opendatadbt.311.littlerock311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('littlerock311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    littlerock_df_cleaned, table_ref, job_config=job_config
).result()  # Make an API request.
